# Read csv 

In [16]:
# dependencies
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
# Read data into dataframe
file = os.path.join('..', 'Raw Data', 'train.csv' )
tweet_df = pd.read_csv(file, encoding='latin-1')
tweet_df.head()

ItemID  Sentiment                                      SentimentText
0       1          0                       is so sad for my APL frie...
1       2          0                     I missed the New Moon trail...
2       3          1                            omg its already 7:30 :O
3       4          0            .. Omgaga. Im sooo  im gunna CRy. I'...
4       5          0           i think mi bf is cheating on me!!!   ...

In [18]:
# check tweet length
tweet_df['pre_clean_len'] = [len(t) for t in tweet_df.SentimentText]
tweet_df.head()

ItemID  Sentiment                                      SentimentText  \
0       1          0                       is so sad for my APL frie...   
1       2          0                     I missed the New Moon trail...   
2       3          1                            omg its already 7:30 :O   
3       4          0            .. Omgaga. Im sooo  im gunna CRy. I'...   
4       5          0           i think mi bf is cheating on me!!!   ...   

   pre_clean_len  
0             61  
1             51  
2             37  
3            132  
4             53

In [19]:
# clean html tags
from bs4 import BeautifulSoup
import string  
import html
import re

# remove url
tweet_df['pre_clean_txt'] = [re.sub(r'http\S+', '', t) for t in tweet_df.SentimentText]

# unescape html characters
tweet_df['pre_clean_txt'] = [html.unescape(t) for t in tweet_df.pre_clean_txt]

# remove string.punctiation: !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
tweet_df['pre_clean_txt'] = [t.translate(str.maketrans('', '', string.punctuation)) for t in tweet_df.pre_clean_txt]

# remove numbers
tweet_df['pre_clean_txt'] = [t.translate(str.maketrans('', '', string.digits)) for t in tweet_df.pre_clean_txt]

# to lower case
tweet_df['pre_clean_txt'] = [t.lower() for t in tweet_df.pre_clean_txt]


#tweet_df['pre_clean_txt'] = [t.translate(str.maketrans('', '', string.punctuation)) for t in tweet_df.pre_clean_txt]
#tweet_df['pre_clean_txt'] = tweet_df.pre_clean_txt.apply(lambda x: x.translate(None, string.digits))

#tweet_df['pre_clean_txt'] = [BeautifulSoup(t['HTML']).get_text().replace('\n',' ') for t in tweet_df.pre_clean_txt]


#tweet_df['pre_clean_txt'] = tweet_df.pre_clean_txt.apply(lambda x: BeautifulSoup(x['HTML']).get_text().replace('\n',' '))
tweet_df.head(100)

ItemID  Sentiment                                      SentimentText  \
0        1          0                       is so sad for my APL frie...   
1        2          0                     I missed the New Moon trail...   
2        3          1                            omg its already 7:30 :O   
3        4          0            .. Omgaga. Im sooo  im gunna CRy. I'...   
4        5          0           i think mi bf is cheating on me!!!   ...   
5        6          0                  or i just worry too much?           
6        7          1                 Juuuuuuuuuuuuuuuuussssst Chillin!!   
7        8          0         Sunny Again        Work Tomorrow  :-|  ...   
8        9          1        handed in my uniform today . i miss you ...   
9       10          1           hmmmm.... i wonder how she my number @-)   
10      11          0                      I must think about positive..   
11      12          1        thanks to all the haters up in my face a...   
12      13          0                     this weekend has sucked so far   
13      14          0             jb isnt showing in australia any more!   
14      15          0                               ok thats it you win.   
15      16          0      &lt;-------- This is the way i feel right ...   
16      17          0      awhhe man.... I'm completely useless rt no...   
17      18          1      Feeling strangely fine. Now I'm gonna go l...   
18      19          0       HUGE roll of thunder just now...SO scary!!!!   
19      20          0      I just cut my beard off. It's only been gr...   
20      21          0                               Very sad about Iran.   
21      22          0                                      wompppp wompp   
22      23          1      You're the only one who can see this cause...   
23      24          0     &lt;---Sad level is 3. I was writing a mass...   
24      25          0     ...  Headed to Hospitol : Had to pull out o...   
25      26          0     BoRinG   ): whats wrong with him??     Plea...   
26      27          0     can't be bothered. i wish i could spend the...   
27      28          0     Feeeling like shit right now. I really want...   
28      29          1              goodbye exams, HELLO ALCOHOL TONIGHT    
29      30          0     I didn't realize it was THAT deep. Geez giv...   
..     ...        ...                                                ...   
70      71          0        I never thought I'd become second choice...   
71      72          0    I think I may be too friendly...lol... o wel...   
72      73          0    I think Manuel (my Basil plant) only has day...   
73      74          0    I wanna be at home @ church...I wonder wht t...   
74      75          0                          i wanna make my own pizza   
75      76          0    i want a 120gb harddrive, or a 37 inch tv, o...   
76      77          0                                       i want a hug   
77      78          0                     I want Miley to tour Australia   
78      79          0    I wanted to sleep in this morning but a mean...   
79      80          0                    i was too slow to get $1 Up tix   
80      81          0    I will send sunshine to Northern Ireland, ar...   
81      82          0    I wish I could go to T4 On The Beach :'(    ...   
82      83          0    i would be so much happier if the walls of m...   
83      84          0    idk wat 2 do who can i trust me im sorry 4 a...   
84      85          0    I'm finding the intercept slope..and banging...   
85      86          1                     I'm really going to bed now...   
86      87          0                             im sick  'cough cough'   
87      88          0    in cab headed to the airport!  going home......   
88      89          0    In case I feel emo in camp (feeling a wee bi...   
89      90          1                                 Jin has a twitter.   
90      91          0                       jonas da

In [20]:
# replace consequent characters, match words against dictionary
from pattern.en import suggest

def reduce_lengthening(text):
   pattern = re.compile(r"(.)\1{2,}")
   return pattern.sub(r"\1\1", text)

def get_clean_word(word):
    word_wlf = suggest(reduce_lengthening(word))[0]
    if(word_wlf[1] > 0.5):
        return word_wlf[0]
    return word

# remove repetitive charecters
#tweet_df['pre_clean_txt'] = [get_clean_word(t) for t in tweet_df.pre_clean_txt]
tweet_df['pre_clean_txt'] = [reduce_lengthening(t) for t in tweet_df.pre_clean_txt]
tweet_df['pre_clean_txt'].head()

0                          is so sad for my apl friend
1                        i missed the new moon trailer
2                                   omg its already  o
3      omgaga im soo  im gunna cry ive been at this...
4                  i think mi bf is cheating on me  tt
Name: pre_clean_txt, dtype: object

In [23]:
# stop words
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#set(stopwords.words('english'))

from nltk.stem.porter import PorterStemmer
#initializes porter
porter = PorterStemmer()

tweet_df['pre_clean_txt'] = [porter.stem(t) for t in tweet_df.pre_clean_txt]
tweet_df['pre_clean_txt'].head()


0                          is so sad for my apl friend
1                          i missed the new moon trail
2                                   omg its already  o
3      omgaga im soo  im gunna cry ive been at this...
4                  i think mi bf is cheating on me  tt
Name: pre_clean_txt, dtype: object

In [24]:

# dictionary of words
import re 
words_df = pd.DataFrame(tweet_df['pre_clean_txt'].values.tolist(), index=tweet_df.index)

word_dict = {}

for ww in words_df[0]:
    for w in re.findall(r'\w+', ww):
        if w in word_dict: 
            word_dict[w] += 1
        else: 
            word_dict[w] = 1

# wc = [t for t in word_dict]

words_df1 = pd.DataFrame.from_dict(word_dict, orient='index')

file = os.path.join('..', 'Output', 'words.csv' )
#wfile = open(file,"w")
#wfile.writelines(sorted(word_dict.keys())
#wfile.close()
    
#sorted(word_dict.keys())
words_df1.head(100)
words_df1.to_csv(file)